In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
x_frame1_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame1_0.5_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame2_0.5_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame3_0.5_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame_noise_0.5_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

659


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
x_adj=np.array(x_adj)

In [11]:
print(x_adj.shape)

(31615, 56, 56)


In [12]:
x_adj=x_adj.reshape(x_adj.shape[0],-1)

In [13]:
print(x_adj.shape)

(31615, 3136)


In [14]:
from sklearn.decomposition import IncrementalPCA
n_components=112
batch_size=2000
ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size)
import time
start_time = time.time()

for i in range(0, x_adj.shape[0], batch_size):
    ipca.partial_fit(x_adj[i:i+batch_size])

X_ipca = ipca.transform(x_adj)
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 27.65400719642639 seconds ---


In [15]:
x_train, x_temp, y_train, y_temp = train_test_split(X_ipca, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [16]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(22130, 112)
(4742, 112)
(4743, 112)


In [17]:
import tensorflow as tf


2023-08-13 10:08:14.576476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 10:08:14.765384: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 10:08:14.769891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [18]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 10:08:39.342303: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 10:08:39.342334: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 10:08:39.342354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0190): /proc/driver/nvidia/version does not exist
2023-08-13 10:08:39.342685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [19]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 112, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 56, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 56, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 28, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 100)               8

In [20]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F7.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [22]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
692/692 [==============================] - 4s 4ms/step - loss: 0.1386 - acc: 0.9554 - val_loss: 0.0143 - val_acc: 0.9975
Epoch 2/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0187 - acc: 0.9970 - val_loss: 0.0108 - val_acc: 0.9979
Epoch 3/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0150 - acc: 0.9971 - val_loss: 0.0090 - val_acc: 0.9983
Epoch 4/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0116 - acc: 0.9977 - val_loss: 0.0071 - val_acc: 0.9985
Epoch 5/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0106 - acc: 0.9980 - val_loss: 0.0074 - val_acc: 0.9983
Epoch 6/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0091 - acc: 0.9981 - val_loss: 0.0104 - val_acc: 0.9979
Epoch 7/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0080 - acc: 0.9983 - val_loss: 0.0095 - val_acc: 0.9983
Epoch 8/30
692/692 [==============================] - 3s 4ms/step - loss: 0.

Epoch 29/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0011 - acc: 0.9995 - val_loss: 0.0174 - val_acc: 0.9985
Epoch 30/30
692/692 [==============================] - 3s 4ms/step - loss: 0.0020 - acc: 0.9993 - val_loss: 0.0153 - val_acc: 0.9983


In [23]:
model3.save_weights('/home/nvekariy/signal_code/New_neural_code_SNR_0.5_h/F7.h5')


In [24]:
model3.load_weights('/home/nvekariy/signal_code/New_neural_code_SNR_0.5_h/F7.h5')


In [25]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

149/149 [==============================] - 0s 2ms/step


In [26]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1006    0    0    0]
 [   1  960    0    1]
 [   0    2  955    0]
 [   2    1    1 1814]]


In [27]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [28]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

99.83133038161502
99.82423182859095
99.83133038161502
99.83152722052769
